In [1]:
import numpy as np
import matplotlib.pyplot as plt

## 6.4 Congestion games

### 6.4.1 Definition

Congestion games are where agents pay a cost for their actions depending on how many other people take the same action. We have a set of resources, and a cost function. The actions are whether or not to claim the resource. The cost is the same across all agents and only depends on the total number of agents claiming the same resource. 

This kind of game appears quite often. As the name suggests one obvious case is when there are two or more different routes from A to B which both take different amounts of time depending on the number of people who choose each. E.g., a highway vs a backroad. But other cases are also common, for instance choosing which bar to go to. In that case there is a cost if there are too may people or too few people (the cost function can be positive!). There seem to be different definitions flying around and in some (like the example above) each actor may only choose 1 resource, but in general there aren't limits. 

### 6.4.2 Computing equilibria

Congestion games have a pure-strategy equilibrium. Not only that, but the equilibrium can be found simply by iteratively updating each player's response for each action. Proof of this is incoming, but first a demonstration:

Say we have two resources A and B. The cost is linear increasing with the number of people:

In [6]:
def MyopicBestResponse(cost_funcs, n_players):
    num_choices = len(cost_funcs)
    strategies = np.random.randint(0,2,(n_players, num_choices))
    any_change = True
    while any_change:
        any_change = False
        for player in range(n_players):
            sum_others = np.sum(strategies,axis=0)-strategies[player]
            for choice in range(num_choices):
                cost = cost_funcs[choice](sum_others[choice]+1)
                if cost>0 and strategies[player,choice]==1:
                    any_change = True
                    strategies[player,choice] = 0
                if cost<0 and strategies[player,choice]==0:
                    any_change = True
                    strategies[player,choice] = 1
    print(strategies)

costA = lambda x: -3 + 2*x
costB = lambda x: -1.5 + x
print("example equilibria")
MyopicBestResponse([costA,costB], 2)

example equilibria
[[1 1]
 [0 0]]


In this case the equilibria are:

$$
\begin{bmatrix}
1 & 1 \\
0 & 0
\end{bmatrix}
\text{or}
\begin{bmatrix}
1 & 0 \\
0 & 1
\end{bmatrix}
\text{or}
\begin{bmatrix}
0 & 1 \\
1 & 0
\end{bmatrix}
\text{or}
\begin{bmatrix}
0 & 0 \\
1 & 1
\end{bmatrix}
$$

I.e., each resource gets 1 person. It gets more interesting with more people.

In [7]:
costA = lambda x: -5 + x
costB = lambda x: -20 + x**2
MyopicBestResponse([costA,costB], 10)

[[0 0]
 [1 1]
 [0 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 0]
 [0 1]
 [1 0]]


Now obviously the idea of MyopicBestResponse is not going to work for all normal-form games, such as the matching pennies problem, but it does work for a subset, called Potential games. 

### 6.4.3 Potential games

A game over $N$ players, $A$ actions, and $u$ utilities is a *potential game* if there is a function $P$ mapping action profiles to a number such that $u_i(a_i,a_{-i})-u_i(a_i^\prime,a_{-i})$ = $P(a_i,a_{-i})-P(a_i^\prime,a_{-i})$. I.e., all agents share the same basic utility function. 

The matching pennies game does not satisfy this constraint:

$
\begin{array}{c|ccc}
\text{} & A & B \\
\hline
\text{A} & 1,-1 & -1,1 \\
\text{B} & -1,1 & 1,-1 \\
\end{array}
$

For instance, say the other player goes A. For the row player the benefit of going A instead of B is 2. But for the column player the benefit is -2.

The staghunt does fit the definition:

$
\begin{array}{c|ccc}
\text{} & C & D \\
\hline
\text{C} & 3,3 & 0,1 \\
\text{D} & 1,0 & 1,1 \\
\end{array}
$

It doesn't matter whether you are row or column the difference is the same conditional on the other player's action.

**Theorem 6.4.5:** Every (finite) potential game has a pure-strategy equilibrium.

Proof is straightforward. Take the argmax of $P$ over all possible action profiles. If it were possible for any player to improve their position they would need to have $u_i(a_i,a_{-i})<u_i(a_i^\prime,a_{-i})$ for some $a_i^\prime$. But this would then imply $P(a_i,a_{-i})<P(a_i^\prime,a_{-i})$, which can't be true, as $P(a)$ was selected as the highest. Simple!

**Theorem 6.4.6:** Every congestion game is a potential game.

Let's definte the potential function $P(a)=\sum_{r\in R} \sum_{j=1}^{\#(r,a)}c_r(j)$. Now we will show that this gives the difference in utilities.

We want to look at $P(a_i,a_{-i})-P(a_i^\prime,a_{-i})$. I.e., we change the i'th player's policy. Expanding the formula gives:

$$\bigg[\sum_{r\in R} \sum_{j=1}^{\#(r,(a_i,a_{-i}))}c_r(j)\bigg]-\bigg[\sum_{r\in R} \sum_{j=1}^{\#(r,(a_i,a_{-i}))}c_r(j)\bigg]$$

We can write this in terms of the current costs of each resource before the current player's turn, PLUS the impact their turn has. The cost of each resource before is $\sum_{j=1}^{\#(r,(a_{-i}))}c_r(j)$. Therefore we get:

$$\bigg[\sum_{r\in R} \sum_{j=1}^{\#(r,(a_{-i}))}c_r(j)+\mathbf{1}_{r\in a_i}c_r(j+1) \bigg]-\bigg[\sum_{r\in R} \sum_{j=1}^{\#(r,(a_{-i}))}c_r(j)+\mathbf{1}_{r\in a_i^\prime}c_r(j+1) \bigg]$$

Which means we can remove the bit in common:

$$\bigg[\sum_{r\in R} \mathbf{1}_{r\in a_i}c_r(j+1) \bigg]-\bigg[\sum_{r\in R} \mathbf{1}_{r\in a_i^\prime}c_r(j+1) \bigg]$$

We can then rewrite as

$$\bigg[\sum_{r\in R|r\in a_i} c_r(j+1) \bigg]-\bigg[\sum_{r\in R|r\in a_i} c_r(j+1) \bigg]$$

Which is exactly the utility functions.

You can also go the other way and show that any potential game is also a congestion game, which is interesting.

Proof that MyopicBestResponse will give a pure equilibrium is now pretty simple. Every step of the iteration is guaranteed to increase $P$.

While this is a nice result, the problem remains PLS-complete (hard) and the time MyopicBestResponse takes might be exponential in certain games.

### 6.4.4 Nonatomic congestion games

A nonatomic congestion game 